<a href="https://colab.research.google.com/github/zombienok/short_quest_generator/blob/dev/quest_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zombienok/short_quest_generator.git
!cd short_quest_generator
!pip install -r /content/short_quest_generator/requirements.txt
!curl -fsSL https://ollama.com/install.sh | sh

Cloning into 'short_quest_generator'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 66.15 KiB | 1.70 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [ ]:
import subprocess
import time
import ollama


#ИСПОЛЬЗУЕМАЯ МОДЕЛЬ
MODEL_TO_USE = "owl/t-lite:latest"

# Запускаем сервер Ollama в фоне (если ещё не запущен)
proc = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ждём, пока сервер запустится (обычно 3-5 секунд)
time.sleep(1)

# Проверяем, работает ли сервер
try:
    ollama.pull(MODEL_TO_USE)
    print(ollama.list())  # Должен вывести список моделей (пустой, если ничего не загружено)
except Exception as e:
    print("Ошибка:", e)

models=[Model(model='owl/t-lite:latest', modified_at=datetime.datetime(2025, 8, 4, 2, 24, 31, 181184, tzinfo=TzInfo(UTC)), digest='52bcfdc27e1b605bb7905cfbfaca2cf5541e85f86c1e1138f0e00d11336242b1', size=4661229593, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


**про константы для генерации структуры**

```COUNT_OF_FIRST_CHOICES = 2``` - количество выборов для стартовой сцены

```CHOICES_AVG = 2```  - максимальное количество выборов для всех остальных сцен

```DEPTH = 3``` - максимальная глубина

ориентировочное количество ```COUNT_OF_SCENES```сцен считается по  формуле:

```COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) +2```
, где +2 - стартовая сцена и финальная

Не рекомендуется менять значение констант (особенно увеличивать), т.к. даже незначительное увеличение приведёт к значительному увеличению количества сцен, что, соответственно, приведёт к переполнению контекста, галлюцинациям (не у Вас, к счастью) и неполной генерации.

In [ ]:
# системный промпт для генерации структуры
messages_strict = [{
    "role": "system",
    "content": "Ты генератор структуры для квеста. Отвечаешь ТОЛЬКО в указанном формате. Никаких комментариев, пояснений, символов *, -, >."
}]

# КОНСТАНТЫ ДЛЯ ГЕНЕРАЦИИ СТРУКТУРЫ
<!--
COUNT_OF_FIRST_CHOICES = 2
CHOICES_AVG = 2
DEPTH = 3
COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) + 2

prompt_structure = f"""
Сгенерируй структуру квеста по следующим правилам:
1. Формат строго:
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум {COUNT_OF_SCENES} уникальных сцен
   - В стартовой сцене {COUNT_OF_FIRST_CHOICES} выбора
   - В нефинальных сценах может быть 1 или {CHOICES_AVG} выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4
"""
print(prompt_structure)
response_structure = ollama.chat(
    model=MODEL_TO_USE,
    messages=messages_strict + [{"role": "user", "content": prompt_structure}],
    options={"temperature":0.3} #ставим температуру пониже для предсказуемости результата
)
structure = response_structure["message"]["content"]
print(structure)


Сгенерируй структуру квеста по следующим правилам:
1. Формат строго: 
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 20 уникальных сцен (не считая финалов/провалов)
   - В стартовой сцене 3 выбора
   - В нефинальных сценах может быть 1 или 2 выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4

1. start_scene: 1 -> scene1, 2 -> scene2, 3 -> scene3
2. scene1: 1 -> scene5, 2 -> scene10, 3 -> fail1
3. scene2: 1 -> scene6, 2 -> final4, 3 -> fail2
4. scene3: 1 -> scene8, 2 -> scene11, 3 -> fail3
5. scene4: 1 -> scene9, 2 -> fail4
6. scene5: 1 -> scene12, 2 -> fail5
7. scene6: 1 -> final5
8. scene7: 1 -> final6
9. scene8: 1 -> 

In [ ]:
# извлекаем информацию из файла
# предполагается, что информация в файле заполнена по шаблону вида Cеттинг: ...\nГлавный герой: ...\nЦель: ...
def extract_info_from_txt(file_path):
    extracted_info = {
        "setting": None,
        "main_charachter": None,
        "goal": None
    }

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("Сеттинг:"):
                extracted_info["setting"] = line.split(":", 1)[1].strip()
            elif line.startswith("Главный герой:"):
                extracted_info["main_charachter"] = line.split(":", 1)[1].strip()
            elif line.startswith("Цель:"):
                extracted_info["goal"] = line.split(":", 1)[1].strip()

    return extracted_info


# импорируем данные из файла
FILE_PATH = r"/content/short_quest_generator/input/input_data3.txt"
extracted_info = extract_info_from_txt(FILE_PATH)
print(extracted_info)


json_strict = f"""
Преврати структуру квеста в JSON строго по этим правилам:
1. Сеттинг: {extracted_info["setting"]}
2. Главный герой: {extracted_info["main_charachter"]}
3. Финальная цель: {extracted_info["goal"]}
Правила генерации:
- Сохрани ВСЕ оригинальные scene_id из структуры и связи между сценами
- Для каждой сцены добавь:
  - "text": описание ситуации (2-3 предложения)
  - "choices": массив выборов (количество соответствует заданной ранее структуре)
- Требования к выборам:
  -выборы должны учитывать описание ГГ, окружающую обстановку и элемент случайности
  -Каждый выбор - конкретное действие (глагол + дополнение)
-"after": немедленное последствие выбора (1 предложение)
- Провалы (failX) должны быть в рамках сеттинга и ГГ
- Финалы (finalX) должны соотноситься с целью: """ + extracted_info["goal"] + """
- Провалы могут быть окончательными (полный провал квеста - "next_scene_id": "fail") или возвратными (возврат к какой-либо из предыдущих сцен - "next_scene_id": "scene1")
- значение next_scene_id для финальных сцен равно "end"
- значение next_scene_id для провальных сцен равно
Формат строго JSON, где сцены выводятся через запятую, без комментариев. Пример формата одной сцены:
{
 "start_scene"{
  "scene_id": "start_scene",
  "text": "...",
  "choices": [
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene1"
      },
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene2"
      }
    ]
  }
}
"""

# если файл некорректный, то прекращаем выполнение
if len(extracted_info.keys()) != 3:
  raise ValueError("Неправильный формат файла")
else:
  messages_strict_json = [{
          "role": "system",
          "content": json_strict}]
  response_json = ollama.chat(
      model=MODEL_TO_USE,
      options = {"temperature": 0.7}, # температуру побольше, чтобы были интереснее результаты
      messages=messages_strict_json + [{"role": "user", "content": structure}],
      format = "json"
  )

  print(response_json["message"]["content"])

{
 "start_scene": {
  "scene_id": "start_scene",
  "text": "Ваш старый друг и бывший ученый-алхимик вызвал вас к себе. Он сообщил о подозрительных событиях в горном селении - там исчезают люди, а местные жители рассказывают страшные истории про появление драконов. Что вы решите сделать?",
  "choices": [
    {
      "text": "Попросить у него помощь и отправиться на разведку",
      "after": "Собрав свои вещи, ты выходишь из старого замка и направляешься к селению.",
      "next_scene_id": "scene1"
    },
    {
      "text": "Настаивать на его помощи для организации экспедиции в горы",
      "after": "Твой бывший знакомый сообщает, что ему нужно время на проверку алгоритмов и материалов.",
      "next_scene_id": "scene2"
    },
    {
      "text": "Отказаться от помощи и отправиться туда самостоятельно",
      "after": "Вы решаете не отвлекать старика своими проблемами. Глубоко вздохнув, ты собираешь свой походный мешок.",
      "next_scene_id": "scene3"
    }
  ]
 },
 
 "scene1": {
   "

In [ ]:
import json
import re

str_json = response_json["message"]["content"]
# Извлечение JSON (на случай, если модель "добавила мусор")
quest_json = json.loads(
    re.search(r'\{.*\}', str_json, flags=re.DOTALL).group(0)
)

print(quest_json)  # Готовый JSON для работы

{'start_scene': {'scene_id': 'start_scene', 'text': 'Ваш старый друг и бывший ученый-алхимик вызвал вас к себе. Он сообщил о подозрительных событиях в горном селении - там исчезают люди, а местные жители рассказывают страшные истории про появление драконов. Что вы решите сделать?', 'choices': [{'text': 'Попросить у него помощь и отправиться на разведку', 'after': 'Собрав свои вещи, ты выходишь из старого замка и направляешься к селению.', 'next_scene_id': 'scene1'}, {'text': 'Настаивать на его помощи для организации экспедиции в горы', 'after': 'Твой бывший знакомый сообщает, что ему нужно время на проверку алгоритмов и материалов.', 'next_scene_id': 'scene2'}, {'text': 'Отказаться от помощи и отправиться туда самостоятельно', 'after': 'Вы решаете не отвлекать старика своими проблемами. Глубоко вздохнув, ты собираешь свой походный мешок.', 'next_scene_id': 'scene3'}]}, 'scene1': {'scene_id': 'scene1', 'text': 'В небольшой деревне люди с опаской смотрят на вас. Ты объясняешь им, что при

In [ ]:
# Запись в файл
with open("quest.json", "w", encoding="utf-8") as file:
    json.dump(quest_json, file, ensure_ascii=False, indent=4)

# Визуализация (опционально, нестабильно)

In [13]:
import json
import logging
from typing import Dict, Any, Optional
from pyvis.network import Network
from IPython.display import display, HTML

# --- Настройка логгирования ---
logging.basicConfig(
    filename='graph_errors.log',
    level=logging.WARNING,
    format='%(asctime)s - %(message)s'
)

# --- Fallback-константы ---
DEFAULT_SCENE_ID = "invalid_scene"
DEFAULT_TEXT = "[Ошибка: текст отсутствует]"
MAX_TEXT_PREVIEW = 30  # Максимальная длина текста в узле

# --- Вспомогательные функции ---
def safe_json_load(raw_json: str) -> Optional[Dict[str, Any]]:
    """Безопасная загрузка JSON с резервными вариантами."""
    try:
        data = json.loads(raw_json)
        if not isinstance(data, dict):
            raise ValueError("JSON root должен быть словарём")
        return data
    except (json.JSONDecodeError, TypeError) as e:
        logging.error(f"CRITICAL: Невалидный JSON. Ошибка: {str(e)}")
        return None
    except Exception as e:
        logging.error(f"UNKNOWN ERROR: {str(e)}")
        return None

def validate_scene(scene_id: str, scene_data: Any) -> bool:
    """Проверка минимальной валидности сцены."""
    if not isinstance(scene_data, dict):
        logging.warning(f"Сцена {scene_id} не является словарём")
        return False

    # Минимально необходимые поля
    if "text" not in scene_data:
        logging.warning(f"Сцена {scene_id} не содержит текста")
        return False

    return True

def get_safe_text(data: Dict, keys: list, default: str = DEFAULT_TEXT) -> str:
    """Рекурсивно получает текст с fallback."""
    current = data
    for key in keys:
        if isinstance(current, dict) and key in current:
            current = current[key]
        else:
            return default
    return str(current)[:MAX_TEXT_PREVIEW] + "..." if current != default else default

def repair_choices(choices: Any) -> list:
    """Ремонт повреждённых вариантов выбора."""
    if not isinstance(choices, list):
        return []

    repaired = []
    for choice in choices:
        if not isinstance(choice, dict):
            continue

        repaired.append({
            "text": choice.get("text", "???"),
            "next_scene_id": choice.get("next_scene_id", DEFAULT_SCENE_ID),
            "after": choice.get("after", "")
        })

    return repaired

# --- Основной пайплайн ---
def visualize_llm_graph(raw_json: [str, dict], output_file: str = "graph.html") -> None:
    # Измененное начало функции:
    if isinstance(raw_json, dict):
        data = raw_json
    else:
        data = safe_json_load(raw_json)
        if not data:
            print("ОШИБКА: Не удалось загрузить JSON")
            return

    # Шаг 2: Подготовка данных
    valid_scenes = set()
    scenes_to_draw = {}

    for scene_id, scene_data in data.items():
        # Автоматический ремонт scene_id
        scene_id = str(scene_id).strip() or f"autogenerated_{len(valid_scenes)}"

        if not validate_scene(scene_id, scene_data):
            continue

        # Ремонт структуры сцены
        repaired_scene = {
            "text": get_safe_text(scene_data, ["text"]),
            "choices": repair_choices(scene_data.get("choices", []))
        }

        scenes_to_draw[scene_id] = repaired_scene
        valid_scenes.add(scene_id)

    # Шаг 3: Визуализация
    net = Network(
        height="800px",
        width="100%",
        directed=True,
        notebook=True,
        cdn_resources="remote"
    )

    # Конфигурация физики (отключаем притяжение узлов)
    net.set_options("""
    {
      "physics": {
        "enabled": true,
        "barnesHut": {
          "gravitationalConstant": -8000,
          "centralGravity": 0,
          "springLength": 200,
          "springConstant": 0.04,
          "damping": 0.09,
          "avoidOverlap": 1
        },
        "minVelocity": 0.75
      },
      "nodes": {
        "font": {
          "size": 14,
          "face": "Tahoma"
        }
      },
      "edges": {
        "smooth": {
          "type": "continuous"
        },
        "font": {
          "size": 12,
          "strokeWidth": 0,
          "align": "middle"
        },
        "color": {
          "inherit": false
        },
        "arrows": {
          "to": {
            "enabled": true,
            "scaleFactor": 0.8
          }
        }
      }
    }
    """)

    # Добавление узлов (только ID сцены в основном отображении)
    for scene_id, scene_data in scenes_to_draw.items():
        net.add_node(
            scene_id,
            label=scene_id,  # Показываем только ID сцены
            title=scene_data["text"],  # Полный текст при наведении
            color="#FFD700" if scene_id == "start_scene" else "#87CEEB",
            font={"size": 16}
        )

    # Добавление связей (текст выборов только при наведении)
    for scene_id, scene_data in scenes_to_draw.items():
        for choice in scene_data["choices"]:
            target = choice["next_scene_id"]
            if target in valid_scenes:  # Только валидные связи
                net.add_edge(
                    scene_id,
                    target,
                    label="",  # Пустая метка по умолчанию
                    title=choice["text"],  # Полный текст при наведении
                    color="#32CD32" if target != DEFAULT_SCENE_ID else "red",
                    arrows="to",
                    smooth=True
                )
            else:
                logging.warning(f"Dead link: {scene_id} -> {target}")

    # Настройка внешнего вида
    net.toggle_hide_edges_on_drag(True)
    net.show_buttons(filter_=['nodes', 'edges'])  # Убрали physics из панели управления

    # Генерация HTML и отображение в Colab
    try:
        # Сохраняем граф во временный файл
        net.save_graph(output_file)

        # Читаем HTML и отображаем его
        with open(output_file, 'r', encoding='utf-8') as f:
            html_content = f.read()

        # Добавляем стиль для более четкого отображения стрелок
        html_content = html_content.replace(
            '</head>',
            '<style>.vis-tooltip {max-width: 300px; word-wrap: break-word;}</style></head>'
        )

        display(HTML(html_content))

    except Exception as e:
        print(f"Ошибка при отображении графа: {e}")
        # Альтернативный метод - показываем через show()
        net.show(output_file)

    print(f"Граф сгенерирован. Невалидных сцен: {len(data) - len(valid_scenes)}")

In [15]:
from google.colab import files
import json

def load_json_file():
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("Файл не был загружен")

    filename = next(iter(uploaded))
    content = uploaded[filename].decode('utf-8')

    try:
        # Пробуем загрузить как JSON
        data = json.loads(content)
        return json.dumps(data)  # Возвращаем строку JSON
    except json.JSONDecodeError:
        # Если не JSON, пробуем как Python-объект
        try:
            data = eval(content, {'__builtins__': None}, {})  # Безопасный eval
            return json.dumps(data)
        except:
            raise ValueError("Файл не является валидным JSON или Python-объектом")

# Использование:
json_string = load_json_file()
visualize_llm_graph(json_string)

Saving quest.json to quest.json


AttributeError: 'dict' object has no attribute 'interaction'